<a href="https://colab.research.google.com/github/mzkhan2000/KG-Embeddings/blob/main/embedding_word_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python program to generate embedding (word vectors) using Word2Vec

# importing necessary modules for embedding
!pip install --upgrade gensim

     |████████████████████████████████| 24.0 MB 80 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
!pip uninstall numpy

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Would remove:
    /usr/bin/f2py
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.19.5


In [4]:
!pip install numpy

     |████████████████████████████████| 15.7 MB 178 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [9]:
# pip install numpy and then hit the RESTART RUNTIME
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import collections
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# check out if google dride mount suceessful 
!ls "/content/drive/My Drive/MonirResearchDatasets"

all1k.ttl.txt
Freebase-GoogleNews-vectors.bin
freebase-vectors-skipgram1000-en.bin
Glove-6B
google_benchmark_data_embeddings_evaluation
google_benchmark_data_embeddings_evaluation_update
google-question-words.txt
Gravity_DBpedia.txt
surround-ga-records


In [5]:
# a funtion for ga-themes extraction from GA-rdf repository separate and return a list all the ga-themes - Monir
def gaThemesExtraction(ga_record):
  gaThemes = []
  with open(ga_record, 'rt') as f:
    data = f.readlines()
  for line in data:
      # check if line contains "ga-themes" sub-string
      if line.__contains__('ga-themes'):
          # split the line contains from "ga-themes" sub-string
          stringTemp = line.split("ga-themes/",1)[1]
          # further split the line contains from "ga-themes" sub-string to delimiter
          stringTemp = stringTemp.split('>')[0]
          gaThemes.append(stringTemp)
  #print(dataLog)
  #print(gaThemes[:9])
  #print(len(gaThemes))
  return gaThemes

In [6]:
# a funtion imput a list of ga-themes and return a list of unique ga-themes and another list of duplicate gaThemes - 
def make_unique_gaThemes(list_all_ga_themes):
  # find a a list of unique ga-themes
  unique_gaThemes = []
  unique_gaThemes = list(dict.fromkeys(gaThemes))
  #print(len(unique_gaThemes))

  # a list of duplicate gaThemes
  duplicate_gaThemes = []
  duplicate_gaThemes = [item for item, count in collections.Counter(gaThemes).items() if count > 1]
  #print(len(duplicate_gaThemes))

  return unique_gaThemes, duplicate_gaThemes

In [11]:
## KG-Embeddings
filename = '/content/drive/My Drive/MonirResearchDatasets/Freebase-GoogleNews-vectors.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [4]:
def embedding_word_clusters(model, list_of_ga_themes, cluster_size):
    keys = list_of_ga_themes
    embedding_model = model
    n = cluster_size
    new_classifier = []
    embedding_clusters = []
    classifier_clusters = []
    for word in keys:
        embeddings = []
        words = []
        # check if a word is fully "OOV" (out of vocabulary) for pre-trained embedding model
        if word in embedding_model.key_to_index:
            # create a new list of classifier
            new_classifier.append(word)
            # find most similar top n words from the pre-trained embedding model
            for similar_word, _ in embedding_model.most_similar(word, topn=n):
                words.append(similar_word)
                embeddings.append(embedding_model[similar_word])
        embedding_clusters.append(embeddings)
        classifier_clusters.append(words)

    return embedding_clusters, classifier_clusters, new_classifier

In [7]:
# to get all the ga-themes from all1K file 
ga_record_datapath = "/content/drive/My Drive/MonirResearchDatasets/surround-ga-records/all1k.ttl.txt"
gaThemes = gaThemesExtraction(ga_record_datapath)
print(gaThemes[:10])
print(len(gaThemes))

['palaeontology', 'geophysics', 'palaeontology', 'geophysics', 'palaeontology', 'stratigraphy', 'palaeontology', 'earthquakes', 'palaeontology', 'palaeontology']
1204


In [10]:
# to get all unique ga-themes
unique_gaThemes, duplicate_gaThemes = make_unique_gaThemes(gaThemes)
print(unique_gaThemes[:100])
#print(duplicate_gaThemes[:100])
print(len(unique_gaThemes))

['palaeontology', 'geophysics', 'stratigraphy', 'earthquakes', 'geomagnetism', 'seismics', 'volcanology', 'groundwater', 'gravity', 'marine', 'magnetics', 'gamma-spectroscopy', 'geology', 'mineral-deposits', 'sedimentary-basins', 'metallogenesis', 'mineragraphy', 'petrography', 'mineral-exploration', 'hydrogeology', 'petroleum-exploration', 'landslides', 'geochemistry', 'petrology', 'marine-survey', 'economic-geology', 'petroleum-geology', 'cartography', 'petroleum-reserves', 'remote-sensing', 'structural-geology', 'continental-margins', 'geochronology', 'bathymetry', 'palaeogeography', 'geoscience-databases', 'data-standards', 'gis', 'landforms', 'regolith', 'metamorphism', 'image-processing', 'satellite-imagery', 'palaeomagnetism', 'geoscience-education', 'tsunamis', 'seismology', 'information-management', 'marine-jurisdiction', 'coasts', 'geomorphology', 'resource-management', 'mineralogy', 'sequence-stratigraphy', 'seismic-sections', 'mapping', 'administrative-boundaries', 'boundar

In [12]:
embedding_clusters, classifier_clusters, new_classifier = embedding_word_clusters(model, unique_gaThemes[:10], 10)

In [13]:
print(classifier_clusters)

[['paleontology', 'paleontological', 'palaeontologists', 'paleoanthropology', 'palaeontologist', 'palaeontological', 'archeology', 'archaeo', 'paleobiology', 'paleontologist'], ['geophysical', 'geophysical_surveys', 'geophysical_survey', 'geological_mapping', 'geochemistry', 'geologic_mapping', 'geochemical', 'Geophysics', 'airborne_geophysics', 'detailed_geological_mapping'], ['volcanics', 'volcanic_stratigraphy', 'lithologies', 'shear_zone', 'stratigraphic', 'gold_mineralization', 'intrusive_rocks', 'intrusives', 'auriferous', 'meta_sediments'], ['quakes', 'temblors', 'earthquake', 'temblor', 'quake', '#.#_magnitude_earthquake', 'tsunamis', 'quakes_measuring', 'seismic_activity', 'aftershocks'], ['Michael_Studinger', 'Cosmic_Background_Explorer_COBE', 'polarimetric', 'Anisotropy_Probe', 'glaciological', 'exoplanetary', 'radar_interferometry', 'meteorological_oceanographic', 'atmospheric_oceanic', 'SeaWiFS'], ['4D_seismic', '3D_seismic_surveys', 'Induced_Polarisation', 'magnetic_geoph

In [14]:
print(new_classifier)

['palaeontology', 'geophysics', 'stratigraphy', 'earthquakes', 'geomagnetism', 'seismics', 'volcanology', 'groundwater', 'gravity', 'marine']


In [15]:
print(classifier_clusters[:2])

[['paleontology', 'paleontological', 'palaeontologists', 'paleoanthropology', 'palaeontologist', 'palaeontological', 'archeology', 'archaeo', 'paleobiology', 'paleontologist'], ['geophysical', 'geophysical_surveys', 'geophysical_survey', 'geological_mapping', 'geochemistry', 'geologic_mapping', 'geochemical', 'Geophysics', 'airborne_geophysics', 'detailed_geological_mapping']]


In [16]:
print(new_classifier[:2])

['palaeontology', 'geophysics']
